In [17]:
import os
import re
import math
import random
import numpy as np
import pandas as pd
from tqdm import tqdm

from sklearn.metrics import mean_squared_error
from sklearn.model_selection import train_test_split

from xgboost import XGBRegressor

from tabulate import tabulate
# from matplotlib import pyplot as plt

DIRPATH = os.path.abspath(".")

In [10]:
transaction = pd.read_csv(DIRPATH + "/data/账户交易信息.csv")
account = pd.read_csv(DIRPATH + "/data/账户静态信息.csv")
label = pd.read_csv(DIRPATH + "/data/训练集标签.csv")
test_df = pd.read_csv(DIRPATH + "/data/test_dataset.csv")

In [21]:
table_shape = []
for dfi in ["transaction", "account", "label", "test_df"]:
    table_shape.append([dfi, eval(dfi).shape])
print(tabulate(table_shape, headers=["Name", "Dataframe Shape"], tablefmt="github"))

| Name        | Dataframe Shape   |
|-------------|-------------------|
| transaction | (816270, 12)      |
| account     | (6000, 5)         |
| label       | (1200, 2)         |
| test_df     | (4800, 1)         |


In [27]:
transaction_info = pd.merge(transaction, account, on="zhdh")
transaction_info.describe()

,jdbj,jyje,zhye,dfmccd,xb,年龄
count,816270.000000,8.162700e+05,8.162700e+05,816270.000000,816270.000000,816270.000000
mean,0.333776,6.053321e+03,3.027487e+04,19.478495,0.361922,35.591966
std,0.471561,1.136937e+05,2.240713e+05,13.920198,0.480557,9.946617
min,0.000000,-5.100000e+05,0.000000e+00,0.000000,0.000000,12.000000
25%,0.000000,1.980000e+01,5.120400e+02,9.000000,0.000000,29.000000
50%,0.000000,1.890000e+02,3.398855e+03,12.000000,0.000000,34.000000
75%,1.000000,1.799910e+03,1.365072e+04,30.000000,1.000000,40.000000
max,1.000000,5.000000e+07,6.324178e+07,113.000000,1.000000,84.000000


In [28]:
df_all = pd.merge(label, transaction_info, on="zhdh")
df_all.describe()

,black_flag,jdbj,jyje,zhye,dfmccd,xb,年龄
count,165311.000000,165311.000000,1.653110e+05,1.653110e+05,165311.000000,165311.000000,165311.000000
mean,0.256141,0.358893,5.281663e+03,3.797571e+04,19.711725,0.394342,36.120397
std,0.436502,0.479677,5.577935e+04,2.601738e+05,13.589711,0.488710,11.059980
min,0.000000,0.000000,-5.000000e+05,0.000000e+00,0.000000,0.000000,12.000000
25%,0.000000,0.000000,1.600000e+01,6.311150e+02,9.000000,0.000000,28.000000
50%,0.000000,0.000000,1.585000e+02,4.314420e+03,16.000000,0.000000,35.000000
75%,1.000000,1.000000,1.547305e+03,2.208128e+04,28.000000,1.000000,41.000000
max,1.000000,1.000000,8.600000e+06,1.029000e+07,97.000000,1.000000,83.000000
